# Entrenamiento y Evaluación de Modelos

## Trabajo Práctico Nro. 2 - Grupo 3

#### Integrantes:
* Ignacio Busso
* Lucas Copes
* Jesica Heit

#### Dataset: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction
* Detalle: contiene datos de la satisfacción de los pasajeros de diferentes vuelos tomando en cuenta multiples aspectos (calidad del servicio, comodidad, limpieza, etc.)
* Target: columna 'satisfaction', para determinar la satisfacción de un pasajero respecto a un vuelo.
* Dimensiones: 25 columnas x 129.880 filas.

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

pd.options.display.max_columns = 0

#Cambios en el estilo de los graficos
plt.style.use('fast')

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [ ]:
# Lectura y concatenación de los .csv
train = pd.read_csv('data/train.csv', index_col=[0])
test = pd.read_csv('data/test.csv', index_col=[0])
all_data = pd.concat([train, test], sort=False)

# Asignamos nuevos nombres a algunas de las columnas
new_column_names = {
    'Gender': 'gender',
    'Customer Type': 'customer_type',
    'Age': 'age',
    'Type of Travel': 'business_travel',
    'Class': 'ticket_class',
    'Flight Distance': 'flight_distance',
    'Inflight wifi service': 'wifi_service',
    'Departure/Arrival time convenient': 'departure_arrival_time_convenient',
    'Ease of Online booking': 'online_booking',
    'Gate location': 'gate_location',
    'Food and drink': 'food_and_drink',
    'Online boarding': 'online_boarding',
    'Seat comfort': 'seat_comfort',
    'Inflight entertainment': 'inflight_entertainment',
    'On-board service': 'onboard_service',
    'Leg room service': 'leg_room',
    'Baggage handling': 'baggage_handling',
    'Checkin service': 'checkin',
    'Inflight service': 'inflight_service',
    'Cleanliness': 'cleanliness',
    'Departure Delay in Minutes': 'departure_delay',
    'Arrival Delay in Minutes': 'arrival_delay',
}

all_data.rename(columns=new_column_names, inplace=True)
all_data.set_index('id', inplace=True)

#### Preprocesado
1. Conversión de variables cualitativas a booleanas cuantitativas.
2. 
    - Limpieza de registros con nulos implícitos en los servicios. Aplica solo a filas con menos de 500 casos.
    - Limpieza de registros NaN en arrival_delays
3. Limpieza de valores extremos en arrival_delays y departure_delays. (~600 registros)
4. Limpieza de valores extremos en flight_distance. Aplica solo a vuelos de más de 4000 unidades de longitud (~75 registros)
5. Feature Engineering creando intervalos para cuatro variables numéricas. (`age`, `flight_distance`, `departure_delay` y `arrival_delay`)

In [ ]:
# Conversión a variables booleanas
all_data['gender'] = all_data['gender'].replace(['Male','Female'],[0,1])
all_data['customer_type'] = all_data['customer_type'].replace(['disloyal Customer','Loyal Customer'],[0,1])
all_data['business_travel'] = all_data['business_travel'].replace(['Personal Travel','Business travel'],[0,1])
all_data['satisfaction'] = all_data['satisfaction'].replace(['neutral or dissatisfied','satisfied'],[0,1])

# Limpieza de filas con pocas (< 500) features de servicios nulas (== 0) y arrivals NaNs
all_data = all_data[
    ~(all_data == 0).gate_location &
    ~(all_data == 0).food_and_drink &
    ~(all_data == 0).seat_comfort & 
    ~(all_data == 0).inflight_entertainment &
    ~(all_data == 0).onboard_service &
    ~(all_data == 0).checkin &
    ~(all_data == 0).inflight_service &
    ~(all_data == 0).cleanliness &
    ~all_data.arrival_delay.isnull()
    ]

# Limpieza de valores extremos en Arrivals y Departures
all_data = all_data.drop(all_data[all_data.arrival_delay > 240].index)
all_data = all_data.drop(all_data[all_data.departure_delay > 240].index)

# Limpieza de valores extremos en flight_distance (> 4000 - 75 registros)
all_data = all_data.drop(all_data[all_data.flight_distance > 4000].index)

# Creación de intervalos para features numéricas
    # Puede ser visto como Binning / Redondeo de números? Feature Engineering?
    # Ver de dejarlo para el final. Ejecutarlo por separado en otro df y comparar resultados.

# ¿Corresponde que los labels tengan como nombre el mayor numero del intervalo? ¿Debería usar un valor calculado en su lugar? (Media/Mediana/Moda)
all_data["age_interval"] = pd.cut(x=all_data['age'], bins=[0,12,18,30,60,80,100], labels=[12,18,30,60,80,100])
all_data["flight_distance_interval"] = pd.cut(x=all_data['flight_distance'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=[500,1000,1500,2000,2500,3000,3500,4000,4500,5000])
all_data["departure_delay_interval"] = pd.cut(x=all_data['departure_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])
all_data["arrival_delay_interval"] = pd.cut(x=all_data['arrival_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])

#### Armado de datasets
- 60% Train         (77k)
- 20% Validation    (25k)
- 20% Test          (25k)

In [ ]:
from sklearn.model_selection import train_test_split

train, not_train = train_test_split(all_data, test_size=0.4, random_state=7)
validation, test = train_test_split(not_train, test_size=0.5, random_state=7)

train.shape, validation.shape, test.shape

#### DataFrameMapper
- Escalar todas las variables numéricas no booleanas con MinMaxScaler (valores entre 0-1).
- Imputar las features con más de 600 nulos con IterativeImputer.

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Imputar y escalar
mapper = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age_interval'], [MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance_interval'], [MinMaxScaler()]),
    (['wifi_service'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['departure_arrival_time_convenient'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['online_booking'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['gate_location'], [MinMaxScaler()]),
    (['food_and_drink'], [MinMaxScaler()]),
    (['online_boarding'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['seat_comfort'], [MinMaxScaler()]),
    (['inflight_entertainment'], [MinMaxScaler()]),
    (['onboard_service'], [MinMaxScaler()]),
    (['leg_room'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['baggage_handling'], [MinMaxScaler()]),
    (['checkin'], [MinMaxScaler()]),
    (['inflight_service'], [MinMaxScaler()]),
    (['cleanliness'], [MinMaxScaler()]),
    (['departure_delay_interval'], [MinMaxScaler()]),
    (['arrival_delay_interval'], [MinMaxScaler()]),
    (['satisfaction'], None)
], df_out=True)

mapper.fit(all_data)
#mfit = mapper.fit(all_data)
#all_data_transformed = mapper.transform(all_data)

In [ ]:
# Transformación de un sample:
sample = all_data.sample(7, random_state=7)
# Sample original:
sample

In [ ]:
# Sample transformado.
mapper.transform(sample)

In [ ]:
# Nombres de los features
mapper.transformed_names_

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer()),
])
# Lo entrenamos con train
pipe.fit(all_data)

In [ ]:
pipe.transform(all_data)

In [ ]:
#Ejemplo IterativeImputer

entradas = np.array((
    (20, 1, 200),
    (10, 1, 0),
    (30, 3, 1),
    (20, 2, 1),
    (10, 2, 23),
))

imputador = IterativeImputer(missing_values=0)
imputador.fit_transform(entradas)